###OGC GetCapabilities

Do they even contain text? Not so much, no.

In [1]:
%matplotlib inline
import pandas as pd
import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from IPython.display import display
from IPython.display import Image

In [2]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))

In [6]:
# average completeness (tokens / expected)
sql = """
select round(avg(json_array_length(tokens) / expected_total::numeric * 100.), 2) as mean_pct,
    round(avg(json_array_length(tokens)), 2) as mean_elements
from ogc_tokens;
"""
df = pd.read_sql(sql, engine)
df

,mean_pct,mean_elements
0,18.59,3.73


So a mean of 20% - of the 19 (wms service) or 22 (ows serviceidentification/serviceprovider) elements we're counting, we get an average of ~4 elements included or 18% of the expected count.

We know that the presence of an element doesn't mean the presence of informative text. The Name element is notorious for this, often defaulting to "WMS" only. Token count is also not great for a lot of these elements. A lot of the Provider information is limited to one token conceptually. 

So what happens if we look at title, abstract and keywords (our high-information elements)?